<a href="https://colab.research.google.com/github/ullasbc-02/natural-language-understanding/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

In [29]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('Tweets_5K.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     5000 non-null   object
 1   text       5000 non-null   object
 2   sentiment  5000 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


In [6]:
raw_tweets = df['text']
sentiments = df['sentiment']

In [7]:
senti_mapping = {
    'negative': -1,
    'neutral': 0,
    'positive': 1
}

labels = df['sentiment'].map(senti_mapping).tolist()

In [17]:
raw_tweets.head(10)

,text
0,"I`d have responded, if I were going"
1,Sooo SAD I will miss you here in San Diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on t..."
5,http://www.dothebouncy.com/smf - some shameles...
6,2am feedings for the baby are fun when he is a...
7,Soooo high
8,Both of you
9,Journey!? Wow... u just became cooler. hehe....


In [16]:
print(labels)

[0, -1, -1, -1, -1, 0, 1, 0, 0, 1, 0, 1, -1, -1, 0, -1, -1, -1, -1, 0, 0, 1, 0, 0, 0, 1, -1, -1, 1, -1, 1, 1, -1, 1, 0, 0, -1, 0, -1, 1, 0, 1, 0, 0, 1, 0, -1, 0, -1, -1, 0, 0, 0, -1, 1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, -1, 0, -1, 0, 1, 1, 0, -1, -1, 1, 0, 0, -1, 1, 1, 0, 0, -1, 0, 1, -1, -1, 0, 1, 1, 0, -1, -1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, -1, -1, 1, 1, 0, 0, 0, -1, -1, -1, 1, 0, 0, 0, 1, 1, 1, -1, -1, 1, 1, 0, 0, 0, -1, 0, -1, -1, -1, -1, 0, -1, 0, 1, 1, 0, 1, -1, 1, -1, 0, 0, 0, -1, 1, -1, 0, 0, -1, 0, -1, 1, 0, 1, 1, 1, 1, -1, 0, 0, -1, -1, 0, -1, 0, 0, 1, 1, -1, 0, 0, 0, -1, 0, -1, 1, 1, 0, -1, -1, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, -1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, 0, -1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, -1, -1, 0, 0, 0, 1, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, -1, -1, 1, -1, 1, 1, 0, -1, 0, -1, 0, -1, 0, 0, 1, -1, -1, 0, 0, 1, 0, 0, 0, 1, 1

In [21]:
basic_preproc_tweets = raw_tweets.str.strip().str.split(" ")

basic_preproc_tweets.head(10)

,text
0,"[I`d, have, responded,, if, I, were, going]"
1,"[Sooo, SAD, I, will, miss, you, here, in, San,..."
2,"[my, boss, is, bullying, me...]"
3,"[what, interview!, leave, me, alone]"
4,"[Sons, of, ****,, why, couldn`t, they, put, th..."
5,"[http://www.dothebouncy.com/smf, -, some, sham..."
6,"[2am, feedings, for, the, baby, are, fun, when..."
7,"[Soooo, high]"
8,"[Both, of, you]"
9,"[Journey!?, Wow..., u, just, became, cooler., ..."


In [32]:
basic_preproc_bow = basic_preproc_tweets.apply(lambda x: pd.Series(x).value_counts()).fillna(0)

In [35]:
basic_preproc_bow.shape

(5000, 15122)

In [33]:
print(basic_preproc_bow)

      I`d  have  responded,   if    I  were  going  Sooo  SAD  will  ...  \
0     1.0   1.0         1.0  1.0  1.0   1.0    1.0   0.0  0.0   0.0  ...   
1     0.0   0.0         0.0  0.0  1.0   0.0    0.0   1.0  1.0   1.0  ...   
2     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
3     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4     0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
...   ...   ...         ...  ...  ...   ...    ...   ...  ...   ...  ...   
4995  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4996  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4997  0.0   0.0         0.0  0.0  0.0   0.0    0.0   0.0  0.0   0.0  ...   
4998  0.0   0.0         0.0  0.0  1.0   0.0    0.0   0.0  0.0   0.0  ...   
4999  0.0   1.0         0.0  0.0  0.0   0.0    0.0   1.0  0.0   0.0  ...   

      reviews  scrapbook.  fiasco  wrapped  midterms   =(  7th  headache-all  \
0      

In [36]:
print(basic_preproc_bow.iloc[1460, 1460])
print(basic_preproc_bow.columns[1460])

0.0
300th


In [34]:
Tweets_train, Tweets_test, Labels_train, Labels_test = train_test_split(basic_preproc_bow, labels, test_size=0.2, shuffle=False)

# Part 2

In [37]:
multi_nb = MultinomialNB(alpha=1.0)
model = multi_nb.fit(Tweets_train, Labels_train)
pred_labels = model.predict(Tweets_test)
accuracy = accuracy_score(Labels_test, pred_labels)
print("Accuracy of NB: ", accuracy)

NameError: name 'MultinomialNB' is not defined

In [38]:
labels_pred_most_common = np.array([Labels_train.value_counts().idxmax()] * Labels_test.count())
accuracy_most_common = accuracy_score(Labels_test, labels_pred_most_common)
print("Accuracy of most common: ", accuracy_most_common)

NameError: name 'np' is not defined

In [ ]:
log_reg = LogisticRegression(max_iter=500)
model = log_reg.fit(Tweets_train, Labels_train)
pred_labels = model.predict(Tweets_test)
accuracy = accuracy_score(Labels_test, pred_labels)
print("Accuracy of Logistic Regression: ", accuracy)